In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [1]:
import detectron2
from pathlib import Path
import random, cv2
import matplotlib.pyplot as plt
import numpy as np
import pycocotools.mask as mask_util
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog, DatasetMapper, build
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger
from detectron2.evaluation.evaluator import DatasetEvaluator
from detectron2.data import transforms as T
from detectron2.engine import BestCheckpointer
from detectron2.checkpoint import DetectionCheckpointer

from detectron2 import solver
import torch
import torch.optim as optim
setup_logger()

<Logger detectron2 (DEBUG)>

In [2]:
dataDir=Path('../input/sartorius-cell-instance-segmentation/')
cfg = get_cfg()
cfg.INPUT.MASK_FORMAT='bitmask'
register_coco_instances('sartorius_train',{}, '../input/sartorius-cell-instance-segmentation-coco/annotations_train.json', dataDir)
register_coco_instances('sartorius_val',{},'../input/sartorius-cell-instance-segmentation-coco/annotations_val.json', dataDir)
metadata = MetadataCatalog.get('sartorius_train')
train_ds = DatasetCatalog.get('sartorius_train')

[12/17 00:35:37 d2.data.datasets.coco]: Loading ../input/sartorius-cell-instance-segmentation-coco/annotations_train.json takes 1.46 seconds.
[12/17 00:35:37 d2.data.datasets.coco]: Loaded 485 images in COCO format from ../input/sartorius-cell-instance-segmentation-coco/annotations_train.json


In [ ]:
# # visualize, do not run in server
# d = train_ds[42]
# img = cv2.imread(d["file_name"])
# visualizer = Visualizer(img[:, :, ::-1], metadata=metadata)
# out = visualizer.draw_dataset_dict(d)
# plt.figure(figsize = (20,15))
# plt.imshow(out.get_image()[:, :, ::-1])

In [3]:
# Taken from https://www.kaggle.com/theoviel/competition-metric-map-iou
def precision_at(threshold, iou):
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1  # Correct objects
    false_positives = np.sum(matches, axis=0) == 0  # Missed objects
    false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
    return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)

def score(pred, targ):
    pred_masks = pred['instances'].pred_masks.cpu().numpy()
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
    enc_targs = list(map(lambda x:x['segmentation'], targ))
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, ious)
        p = tp / (tp + fp + fn)
        prec.append(p)
    return np.mean(prec)

class MAPIOUEvaluator(DatasetEvaluator):
    def __init__(self, dataset_name):
        dataset_dicts = DatasetCatalog.get(dataset_name)
        self.annotations_cache = {item['image_id']:item['annotations'] for item in dataset_dicts}
            
    def reset(self):
        self.scores = []

    def process(self, inputs, outputs):
        for inp, out in zip(inputs, outputs):
            if len(out['instances']) == 0:
                self.scores.append(0)    
            else:
                targ = self.annotations_cache[inp['image_id']]
                self.scores.append(score(out, targ))

    def evaluate(self):
        return {"MaP IoU": np.mean(self.scores)}

class Trainer(DefaultTrainer):
    
    # @classmethod
    # def build_train_loader(cls, cfg):
    #     return build_detection_train_loader(cfg, mapper=DatasetMapper(cfg, is_train=True, augmentations=[
    #             T.ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'),               
	# 	        T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
    #             T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
    #         ]))
    
    # @classmethod
    # def build_optimizer(cfg, model):
    
        
    #     return torch.optim.Adadelta(
    #         model.paramaters(),
    #         lr=cfg.SOLVER.BASE_LR,
    #     )

    # @classmethod
    # def build_train_loader(cls, cfg):
    #     return build.build_detection_train_loader(cfg, mapper=DatasetMapper(cfg, is_train=True, augmentations=[
    #             T.RandomBrightness(0.9, 1.1),
    #             T.RandomContrast(0.9, 1.1),
    #             T.RandomSaturation(0.9, 1.1),
    #             T.RandomLighting(0.9),
    #             T.ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'),               
	# 	        T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
    #             T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
    #         ]))
            
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return MAPIOUEvaluator(dataset_name)

    def build_hooks(self):

        # copy of cfg
        cfg = self.cfg.clone()

        # build the original model hooks
        hooks = super().build_hooks()

        # add the best checkpointer hook
        hooks.insert(-1, BestCheckpointer(cfg.TEST.EVAL_PERIOD, 
                                         DetectionCheckpointer(self.model, cfg.OUTPUT_DIR),
                                         "MaP IoU",
                                         "max",
                                         ))
        return hooks

    
    

In [4]:
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("sartorius_train",)
cfg.DATASETS.TEST = ("sartorius_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = './model_best.pth'  # Let training initialize from pretrained model
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 10000    
cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR" 
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .5
cfg.TEST.EVAL_PERIOD = len(DatasetCatalog.get('sartorius_train')) // cfg.SOLVER.IMS_PER_BATCH  # Once per epoch

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = Trainer(cfg) 
trainer.resume_or_load(resume=False)

[12/17 00:35:46 d2.data.datasets.coco]: Loading ../input/sartorius-cell-instance-segmentation-coco/annotations_train.json takes 1.18 seconds.
[12/17 00:35:46 d2.data.datasets.coco]: Loaded 485 images in COCO format from ../input/sartorius-cell-instance-segmentation-coco/annotations_train.json
[12/17 00:35:52 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (9, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (9,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (32, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (32,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (8, 256, 1, 1) in the checkpoint but (3, 256, 1, 1) in th

In [5]:
trainer.train()

[12/17 00:35:57 d2.engine.train_loop]: Starting training from iteration 0


/home/gaoyiyuan/miniconda3/envs/NMDA_gaoyy/lib/python3.9/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/home/gaoyiyuan/miniconda3/envs/NMDA_gaoyy/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272068185/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[12/17 00:36:30 d2.utils.events]:  eta: 2:01:27  iter: 19  total_loss: 3.042  loss_cls: 1.427  loss_box_reg: 0.2707  loss_mask: 0.695  loss_rpn_cls: 0.3436  loss_rpn_loc: 0.2637  time: 1.5460  data_time: 1.2835  lr: 1.9516e-05  max_mem: 4664M
[12/17 00:36:55 d2.utils.events]:  eta: 1:44:54  iter: 39  total_loss: 2.921  loss_cls: 1.28  loss_box_reg: 0.4274  loss_mask: 0.6836  loss_rpn_cls: 0.2449  loss_rpn_loc: 0.2322  time: 1.3895  data_time: 0.9491  lr: 3.9007e-05  max_mem: 5716M
[12/17 00:37:21 d2.utils.events]:  eta: 1:39:50  iter: 59  total_loss: 2.422  loss_cls: 0.9201  loss_box_reg: 0.2959  loss_mask: 0.6552  loss_rpn_cls: 0.2479  loss_rpn_loc: 0.2685  time: 1.3576  data_time: 0.9752  lr: 5.8497e-05  max_mem: 5716M
[12/17 00:37:43 d2.utils.events]:  eta: 1:31:23  iter: 79  total_loss: 2.103  loss_cls: 0.6825  loss_box_reg: 0.341  loss_mask: 0.6067  loss_rpn_cls: 0.2125  loss_rpn_loc: 0.2432  time: 1.2932  data_time: 0.7623  lr: 7.7988e-05  max_mem: 5716M
[12/17 00:38:14 d2.utils.

In [ ]:
# # visualize, do not run in server
# cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
# predictor = DefaultPredictor(cfg)
# dataset_dicts = DatasetCatalog.get('sartorius_val')
# outs = []
# for d in random.sample(dataset_dicts, 3):    
#     im = cv2.imread(d["file_name"])
#     outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
#     v = Visualizer(im[:, :, ::-1],
#                    metadata = MetadataCatalog.get('sartorius_train'), 
                    
#                    instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
#     )
#     out_pred = v.draw_instance_predictions(outputs["instances"].to("cpu"))
#     visualizer = Visualizer(im[:, :, ::-1], metadata=MetadataCatalog.get('sartorius_train'))
#     out_target = visualizer.draw_dataset_dict(d)
#     outs.append(out_pred)
#     outs.append(out_target)
# _,axs = plt.subplots(len(outs)//2,2,figsize=(40,45))
# for ax, out in zip(axs.reshape(-1), outs):
#     ax.imshow(out.get_image()[:, :, ::-1])